In [1]:
# %%
!pip install pyinputplus


import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip # Import pyinputplus after installing
from datetime import datetime, timedelta
from typing import Optional, Dict, List

class WeatherFetcher:
    """Handles weather data retrieval from wttr.in API"""

    BASE_URL = "https://wttr.in/{location}?format=j1"

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'Weather Advisor App'})

    def get_weather(self, location: str, forecast_days: int = 5) -> Optional[Dict]:
        """Fetch weather data for specified location"""
        try:
            response = self.session.get(self.BASE_URL.format(location=location))
            response.raise_for_status()
            return self._structure_data(response.json(), forecast_days)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching weather data: {e}")
            return None

    def _structure_data(self, raw_data: Dict, forecast_days: int) -> Dict:
        """Structure raw API data into standardized format"""
        current = raw_data["current_condition"][0]
        return {
            "current": {
                "temp_C": current["temp_C"],
                "weatherDesc": current["weatherDesc"][0]["value"],
                "precipMM": current["precipMM"],
                "humidity": current["humidity"],
                "windspeedKmph": current["windspeedKmph"]
            },
            "forecast": [
                self._structure_forecast_day(day)
                for day in raw_data["weather"][:forecast_days]
            ]
        }

    def _structure_forecast_day(self, day: Dict) -> Dict:
        """Structure individual forecast day data"""
        return {
            "date": day["date"],
            "maxtempC": day["maxtempC"],
            "mintempC": day["mintempC"],
            "avgtempC": day["avgtempC"],
            "precipMM": day["hourly"][0]["precipMM"],
            "weatherDesc": day["hourly"][0]["weatherDesc"][0]["value"]
        }

class WeatherParser:
    """Handles natural language processing of weather questions"""

    TIME_KEYWORDS = {
        "today": ["today", "now", "current"],
        "tomorrow": ["tomorrow", "next day"],
        "weekend": ["weekend", "saturday", "sunday"]
    }

    ATTRIBUTE_KEYWORDS = {
        "temperature": ["temp", "hot", "cold", "warm", "freezing"],
        "precipitation": ["rain", "snow", "precipitation", "shower"],
        "humidity": ["humid", "moisture"],
        "wind": ["wind", "breezy", "gust"]
    }

    def parse_question(self, question: str) -> Dict:
        """Extract location, time period, and attribute from question"""
        return {
            "location": self._extract_location(question),
            "time": self._extract_time_period(question),
            "attribute": self._extract_attribute(question)
        }

    def _extract_location(self, question: str) -> Optional[str]:
        """Extract location using pattern matching"""
        patterns = [" in ", " at ", " for "]
        for pattern in patterns:
            if pattern in question.lower():
                return question.lower().split(pattern)[1].split(" ")[0].strip("?,")
        return None

    def _extract_time_period(self, question: str) -> str:
        """Identify time period in question"""
        question_lower = question.lower()
        for time_period, keywords in self.TIME_KEYWORDS.items():
            if any(keyword in question_lower for keyword in keywords):
                return time_period
        return "today"

    def _extract_attribute(self, question: str) -> str:
        """Identify weather attribute in question"""
        question_lower = question.lower()
        for attribute, keywords in self.ATTRIBUTE_KEYWORDS.items():
            if any(keyword in question_lower for keyword in keywords):
                return attribute
        return "general"

class WeatherVisualizer:
    """Handles creation of weather visualizations"""

    def show_temperature_trend(self, weather_data: Dict) -> None:
        """Display temperature forecast visualization"""
        if not self._validate_forecast_data(weather_data):
            return

        dates = [day["date"] for day in weather_data["forecast"]]
        max_temps = [float(day["maxtempC"]) for day in weather_data["forecast"]]
        min_temps = [float(day["mintempC"]) for day in weather_data["forecast"]]

        plt.figure(figsize=(10, 5))
        plt.plot(dates, max_temps, 'r-', label='Max Temp (°C)')
        plt.plot(dates, min_temps, 'b-', label='Min Temp (°C)')
        plt.fill_between(dates, max_temps, min_temps, color='gray', alpha=0.1)
        self._format_plot("Temperature Forecast", "Date", "Temperature (°C)")
        plt.show()

    def show_precipitation(self, weather_data: Dict) -> None:
        """Display precipitation forecast visualization"""
        if not self._validate_forecast_data(weather_data):
            return

        dates = [day["date"] for day in weather_data["forecast"]]
        precip = [float(day["precipMM"]) for day in weather_data["forecast"]]

        plt.figure(figsize=(10, 5))
        bars = plt.bar(dates, precip, color='blue', alpha=0.7)
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height}mm', ha='center', va='bottom')
        self._format_plot("Precipitation Forecast", "Date", "Precipitation (mm)")
        plt.show()

    def _format_plot(self, title: str, x_label: str, y_label: str) -> None:
        """Common plot formatting logic"""
        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.legend()

    def _validate_forecast_data(self, weather_data: Dict) -> bool:
        """Validate forecast data availability"""
        if not weather_data or "forecast" not in weather_data:
            print("No forecast data available for visualization.")
            return False
        return True

class WeatherInterface:
    """Handles user interaction and application flow"""

    def __init__(self):
        self.fetcher = WeatherFetcher()
        self.parser = WeatherParser()
        self.visualizer = WeatherVisualizer()
        self.current_location = None
        self.weather_data = None

    def run(self) -> None:
        """Main application loop"""
        print("\n🌤️ Welcome to Weather Advisor! 🌤️")
        print("Get weather forecasts and ask questions in natural language.\n")

        while True:
            choice = pyip.inputMenu(
                ["Set Location", "Current Weather", "Ask a Question",
                 "View Forecast", "Visualizations", "Exit"],
                prompt="\nChoose an option:\n",
                numbered=True
            )

            {
                "Set Location": self._handle_location,
                "Current Weather": self._show_current_weather,
                "Ask a Question": self._handle_question,
                "View Forecast": self._show_forecast,
                "Visualizations": self._handle_visualizations,
                "Exit": self._exit_app
            }[choice]()

    def _handle_location(self) -> None:
        """Set new location handler"""
        self.current_location = pyip.inputStr("Enter location (e.g., London, New York): ")
        self.weather_data = self.fetcher.get_weather(self.current_location)
        if self.weather_data:
            print(f"\n✅ Weather data loaded for {self.current_location}")
        else:
            print("\n❌ Could not fetch weather data. Please try another location.")
            self.current_location = None

    def _show_current_weather(self) -> None:
        """Display current weather summary"""
        if not self._validate_data():
            return

        current = self.weather_data["current"]
        print(f"\n🌦️ Current Weather in {self.current_location}:")
        print(f"  - Condition: {current['weatherDesc']}")
        print(f"  - Temperature: {current['temp_C']}°C")
        print(f"  - Precipitation: {current['precipMM']}mm")
        print(f"  - Humidity: {current['humidity']}%")
        print(f"  - Wind: {current['windspeedKmph']} km/h\n")

    def _handle_question(self) -> None:
        """Process user questions"""
        if not self._validate_data():
            return

        question = pyip.inputStr("\nAsk a weather question (e.g., 'Will it rain tomorrow?'): ")
        parsed = self.parser.parse_question(question)
        parsed["location"] = parsed["location"] or self.current_location

        response = self._generate_response(parsed)
        print(f"\n🤖 Weather Advisor: {response}")

    def _generate_response(self, parsed: Dict) -> str:
        """Generate response to parsed question"""
        time_period = parsed["time"]
        attribute = parsed["attribute"]

        if time_period == "today":
            data = self.weather_data["current"]
        else:
            data = self._get_forecast_data(time_period)

        if not data:
            return "Unable to find forecast data for that time period."

        return {
            "temperature": lambda: f"The temperature in {parsed['location']} {time_period} is {data['temp_C' if time_period == 'today' else 'avgtempC']}°C.",
            "precipitation": lambda: self._get_precipitation_response(data, parsed),
            "humidity": lambda: f"The humidity in {parsed['location']} {time_period} is {data['humidity']}%.",
            "wind": lambda: f"The wind speed in {parsed['location']} {time_period} is {data['windspeedKmph']} km/h.",
            "general": lambda: f"The weather in {parsed['location']} {time_period}: {data['weatherDesc']}."
        }.get(attribute, lambda: "I'm not sure how to answer that.")()

    def _get_forecast_data(self, time_period: str) -> Optional[Dict]:
        """Retrieve forecast data for specific time period"""
        if time_period == "tomorrow":
            return self.weather_data["forecast"][0] if len(self.weather_data["forecast"]) >= 1 else None
        elif time_period == "weekend":
            return next((day for day in self.weather_data["forecast"]
                        if self._is_weekend(day["date"])), None)
        return None

    def _is_weekend(self, date_str: str) -> bool:
        """Check if date falls on a weekend"""
        date = datetime.strptime(date_str, "%Y-%m-%d")
        return date.weekday() >= 5

    def _get_precipitation_response(self, data: Dict, parsed: Dict) -> str:
        """Generate precipitation-specific response"""
        precip = float(data["precipMM"])
        if precip > 5:
            return f"There will be significant rain ({precip}mm) in {parsed['location']} {parsed['time']}."
        elif precip > 0:
            return f"There might be some rain ({precip}mm) in {parsed['location']} {parsed['time']}."
        return f"No precipitation expected in {parsed['location']} {parsed['time']}."

    def _show_forecast(self) -> None:
        """Display 5-day forecast"""
        if not self._validate_data():
            return

        print(f"\n📅 5-Day Forecast for {self.current_location}:")
        for day in self.weather_data["forecast"]:
            print(f"\n  {day['date']}:")
            print(f"  - Condition: {day['weatherDesc']}")
            print(f"  - Max Temp: {day['maxtempC']}°C")
            print(f"  - Min Temp: {day['mintempC']}°C")
            print(f"  - Precipitation: {day['precipMM']}mm")

    def _handle_visualizations(self) -> None:
        """Handle visualization selection"""
        if not self._validate_data():
            return

        viz_choice = pyip.inputMenu(
            ["Temperature Trends", "Precipitation Forecast", "Both"],
            prompt="\n📊 Choose visualization:\n",
            numbered=True
        )

        if viz_choice in ["Temperature Trends", "Both"]:
            self.visualizer.show_temperature_trend(self.weather_data)
        if viz_choice in ["Precipitation Forecast", "Both"]:
            self.visualizer.show_precipitation(self.weather_data)

    def _exit_app(self) -> None:
        """Handle application exit"""
        print("\nThank you for using Weather Advisor! Goodbye. 👋")
        exit()

    def _validate_data(self) -> bool:
        """Validate weather data availability"""
        if not self.current_location or not self.weather_data:
            print("\nPlease set a location first.")
            return False
        return True

if __name__ == "__main__":
    app = WeatherInterface()
    app.run()

KeyboardInterrupt: Interrupted by user